In [1]:
import os

In [2]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal/research'

In [3]:
os.chdir('../.')

In [4]:
%pwd

'/Users/bhikipallai/Desktop/Projects/Chicken_fecal'

In [5]:
import dagshub
dagshub.init(repo_owner='Vicky7873', repo_name='Chicken_fecal', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as Vicky7873

Initialized MLflow to track repo "Vicky7873/Chicken_fecal"

Repository Vicky7873/Chicken_fecal initialized!

2024/09/22 22:52:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run righteous-shrike-683 at: https://dagshub.com/Vicky7873/Chicken_fecal.mlflow/#/experiments/0/runs/4a91fdc0e84a48649c6d9097d5be86eb.
2024/09/22 22:52:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Vicky7873/Chicken_fecal.mlflow/#/experiments/0.


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    score: Path
    mlflow_uri: str 
    all_params: dict

In [7]:
from src.constants import *
from src.utils.common import read_yaml,save_json,create_directories

2024-09-22 22:52:49,083 - INFO - start of looging


In [8]:
class ConfigurationManagerone:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.data_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.Model_Evaluation
        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            score = config.score,
            mlflow_uri='https://dagshub.com/Vicky7873/Chicken_fecal.mlflow',
            all_params=self.params
        )
        return model_evaluation_config

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from src.components.data_aug import DataAugmentation
from src.entity import DataAugConfig
from src.config.configuration import ConfigurationManager

In [10]:
import mlflow
import mlflow.keras

In [17]:
import mlflow.entities


class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig,evl_config:DataAugmentation):
        self.config = config
        self.data_config = evl_config

    def model_load(self):
        model = load_model(self.config.model_path)
        return model

    def model_evaluate(self):
        model = self.model_load()
        obj = DataAugmentation(self.data_config)
        train_data, test_data, val_data = obj.augment_data()

        self.train_acc = model.evaluate(train_data, verbose=0)[1]
        self.test_acc = model.evaluate(test_data, verbose=0)[1]
        
        print(f"Training Accuracy: {self.train_acc * 100:.2f}%")
        print(f"Testing Accuracy: {self.test_acc * 100:.2f}%")

    def save_score(self):
        scores = {
            "train_acc": self.train_acc,
            "test_acc": self.test_acc
        }
        json_path = self.config.score
        save_json(path = Path(json_path), data = scores)

    def log_model_mlfow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_experiment("Mlflow Bhiki")

        with mlflow.start_run():
            mlflow.log_metric("train_acc", self.train_acc)
            mlflow.log_artifact(self.config.score)
            

            for param_name, param_value in self.config.all_params.items():
                mlflow.log_param(param_name, param_value)

            # Register the model in the MLflow Model Registry
            model = self.model_load()
            mlflow.keras.log_model(model, "model", registered_model_name="VGG16Model")
            
            # Logging the model as an artifact
            mlflow.keras.log_model(model, "model")
            

In [18]:
try:
    config = ConfigurationManagerone()
    eval_config = config.get_model_evaluation_config()

    data_confog = ConfigurationManager()
    aug_config = data_confog.get_data_aug()

    model_eval = ModelEvaluation(config = eval_config, evl_config = aug_config)
    model_eval.model_evaluate()
    model_eval.save_score()
    model_eval.log_model_mlfow()
except Exception as e:
    raise e

2024-09-22 23:02:22,079 - INFO - Yaml file: config/config.yaml loaded successfully
2024-09-22 23:02:22,084 - INFO - Yaml file: params.yaml loaded successfully
2024-09-22 23:02:22,085 - INFO - created directory at: data
2024-09-22 23:02:22,086 - INFO - created directory at: data/model_evaluation
2024-09-22 23:02:22,091 - INFO - Yaml file: config/config.yaml loaded successfully
2024-09-22 23:02:22,094 - INFO - Yaml file: params.yaml loaded successfully
2024-09-22 23:02:22,095 - INFO - created directory at: data
Found 272 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 58 images belonging to 2 classes.
Training Accuracy: 81.62%
Testing Accuracy: 98.33%
2024-09-22 23:03:00,551 - INFO - json file saved at: data/model_evaluation/chicken_score.json


2024/09/22 23:03:05 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/22 23:03:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/09/22 23:03:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
2024/09/22 23:03:21 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2024/09/22 23:03:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/09/22 23:03:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run silent-new